In [65]:
import os
import sionna as sn

# Import TensorFlow and NumPy
import tensorflow as tf

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import time
from sionna.utils.plotting import PlotBER, sim_ber

# For the implementation of the Keras models
from tensorflow.keras import Model

import wandb


In [54]:
class CodedSystemAWGN(Model): # Inherits from Keras Model
    def __init__(self, num_bits_per_symbol, n, coderate):
        super().__init__() # Must call the Keras model initializer

        self.num_bits_per_symbol = num_bits_per_symbol
        self.n = n
        self.k = int(n*coderate)
        self.coderate = coderate
        self.constellation = sn.mapping.Constellation("qam", self.num_bits_per_symbol)

        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation)

        self.binary_source = sn.utils.BinarySource()
        self.awgn_channel = sn.channel.AWGN()

        self.encoder = sn.fec.ldpc.LDPC5GEncoder(self.k, self.n)
        self.decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=True)

    #@tf.function # activate graph execution to speed things up
    def __call__(self, batch_size, ebno_db):
        no = sn.utils.ebnodb2no(ebno_db, num_bits_per_symbol=self.num_bits_per_symbol, coderate=self.coderate)

        bits = self.binary_source([batch_size, self.k])
        codewords = self.encoder(bits)
        x = self.mapper(codewords)
        y = self.awgn_channel([x, no])
        llr = self.demapper([y,no])
        bits_hat = self.decoder(llr)
        return bits, bits_hat

In [57]:
CODERATE = 0.5
BATCH_SIZE = 2000
NUM_BITS_PER_SYMBOL = 2

model_coded_awgn = CodedSystemAWGN(num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                                   n=128,
                                   coderate=CODERATE)

snr_db = np.linspace(1, 5, 5)
ber, bler = sim_ber(model_coded_awgn, ebno_dbs=snr_db, batch_size=BATCH_SIZE, num_target_block_errors=5000, max_mc_iter=100)

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
      1.0 | 8.7430e-02 | 5.0220e-01 |       55955 |      640000 |         5022 |       10000 |         5.1 |reached target block errors
      2.0 | 2.5313e-02 | 1.5412e-01 |       55081 |     2176000 |         5240 |       34000 |        17.1 |reached target block errors
      3.0 | 3.4559e-03 | 2.1950e-02 |       44236 |    12800000 |         4390 |      200000 |       100.9 |reached max iter       
      4.0 | 2.2906e-04 | 1.4650e-03 |        2932 |    12800000 |          293 |      200000 |       101.3 |reached max iter       
      5.0 | 3.4375e-06 | 2.5000e-05 |          44 |    12800000 |            5 |      200000 |       100.9 |reached max iter       


In [58]:
ber

<tf.Tensor: shape=(5,), dtype=float64, numpy=
array([8.74296875e-02, 2.53129596e-02, 3.45593750e-03, 2.29062500e-04,
       3.43750000e-06])>

In [59]:
bler

<tf.Tensor: shape=(5,), dtype=float64, numpy=
array([5.02200000e-01, 1.54117647e-01, 2.19500000e-02, 1.46500000e-03,
       2.50000000e-05])>

In [67]:
wandb.login()

AttributeError: module 'wandb' has no attribute 'login'

In [69]:
# Initialize Weights and Biases
wandb.login()

# tell wandb to get started
with wandb.init(
    project="Joint-detection-and-decoding-in-short-packet-communications",
):

    # define our custom x axis metric
    wandb.define_metric("custom_step")

    # define which metrics will be plotted against it
    # first plot for BER
    wandb.define_metric(
        f"BER v.s. SNR (AWGN)",
        step_metric="custom_step",
    )

    # second plot for BLER
    wandb.define_metric(
        f"BLER v.s. SNR (AWGN)",
        step_metric="custom_step",
    )

    for i in range(len(snr_db)):

        log_dict = {
            f"BER v.s. SNR (AWGN)": ber[i],
            "custom_step": snr_db[i],
        }
        wandb.log(log_dict)

        log_dict = {
            f"BLER v.s. SNR (AWGN)": bler[i],
            "custom_step": snr_db[i],
        }
        wandb.log(log_dict)

AttributeError: module 'wandb' has no attribute 'login'